In [1]:

import os
import glob
import sys
sys.path.append('../../')
from tifffile import imread, imwrite
import numpy as np
from tqdm import tqdm
from vollseg import Augmentation2D
from vollseg.helpers import crappify_flou_G_P
from pathlib import Path

ImportError: cannot import name 'Augmentation2D' from 'vollseg' (/home/sancere/anaconda3/envs/tensorflowGPU/lib/python3.7/site-packages/vollseg/__init__.py)

In [ ]:
ImageDir =  '/home/sancere/VKepler/CurieTrainingDatasets/Raw/'
LabelDir = '/home/sancere/VKepler/CurieTrainingDatasets/RealMask/'



In [ ]:
Raw_path = os.path.join(imagedir, '*tif')
filesRaw = glob.glob(Raw_path)
filesRaw.sort


RealRaw_path = os.path.join(LabelDir, '*tif')
RealfilesRaw = glob.glob(RealRaw_path)
RealfilesRaw.sort
mu = 0
sigma = 5
for fname in filesRaw:


           for Realfname in RealfilesRaw:

                  name = os.path.basename(os.path.splitext(fname)[0])
                  realname = os.path.basename(os.path.splitext(Realfname)[0])

                  if ( name == realname): 

                            #Read image        
                            image = imread(fname)
                            y = imread(Realfname)
                            #Get Name
                            x = image
                            crappify_flou_G_P(x, y, mu, sigma, imagedir, labeldir, name)
                           